In [2]:
import sys
import random
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import lambertw
import seaborn as sns
import pickle
import warnings

import pyximport
pyximport.install(setup_args={"include_dirs":np.get_include()},
                  reload_support=True)
import implementations as f
import clip as clip

/Users/adam/miniconda3/lib/python3.11/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /Users/adam/GitHub/caslb-experiments/clip.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /Users/adam/.pyxbld/temp.macosx-10.9-x86_64-cpython-311/Users/adam/GitHub/caslb-experiments/clip.c:1240:
In file included from /Users/adam/miniconda3/lib/python3.11/site-packages/numpy/core/include/numpy/arrayobject.h:5:
In file included from /Users/adam/miniconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/adam/miniconda3/lib/python3.11/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929:
/Users/adam/miniconda3/lib/python3.11/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY

In [3]:
# solve for gamma
L = 1
U = 500
beta = 20
epsilon = 3
gamma = clip.solve_gamma((1+epsilon), U, L, beta)

In [4]:
print(gamma)

(465.6095222449376+0j)


In [5]:
epsilon = 0.0001
gamma = clip.solve_gamma((1+epsilon), U, L, beta)
print(gamma)

(499.9988525004798+0j)
